In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv('DATA/netflix_titles.csv')

In [3]:
model = TfidfVectorizer(stop_words='english')
df['description'] = df['description'].fillna('')
feature_matrix = model.fit_transform(df['description'])

In [4]:
feature_matrix.shape

(7787, 17905)

In [5]:
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [6]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [7]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [8]:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

In [9]:
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

In [10]:
#!pip install lightfm

In [11]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [12]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [13]:
df = pd.read_csv('data/ratings.csv')

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape)

(785404, 3)


In [15]:
n_books=len(df.book_id.unique())

In [16]:
n_books

10000

In [17]:
n_users=len(df.user_id.unique())
n_users

53424

In [18]:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


In [19]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [20]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [21]:
conc = Concatenate()([book_vec, user_vec])

In [22]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [23]:
model2 = Model([user_input, book_input], out)

In [24]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [25]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5


24544/24544 [==============================] - 361s 15ms/step - loss: 0.7997
Epoch 2/5
24544/24544 [==============================] - 342s 14ms/step - loss: 0.6868
Epoch 3/5
24544/24544 [==============================] - 349s 14ms/step - loss: 0.6550
Epoch 4/5
24544/24544 [==============================] - 353s 14ms/step - loss: 0.6298
Epoch 5/5
24544/24544 [==============================] - 341s 14ms/step - loss: 0.6061


In [26]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 21s 3ms/step - loss: 0.7099


0.7098538279533386

In [27]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 351s 14ms/step - loss: 0.6095
Epoch 2/10
24544/24544 [==============================] - 337s 14ms/step - loss: 0.5692
Epoch 3/10
24544/24544 [==============================] - 336s 14ms/step - loss: 0.5489
Epoch 4/10
24544/24544 [==============================] - 340s 14ms/step - loss: 0.5324
Epoch 5/10
24544/24544 [==============================] - 429s 17ms/step - loss: 0.5190
Epoch 6/10
24544/24544 [==============================] - 344s 14ms/step - loss: 0.5085
Epoch 7/10
24544/24544 [==============================] - 339s 14ms/step - loss: 0.4993
Epoch 8/10
24544/24544 [==============================] - 338s 14ms/step - loss: 0.4918
Epoch 9/10
24544/24544 [==============================] - 378s 15ms/step - loss: 0.4847
Epoch 10/10
6136/6136 [==============================] - 40s 6ms/step - loss: 0.7703


0.7703008055686951